In [ ]:
from qem.image_fitting import ImageModelFitting
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import hyperspy.api as hs
%matplotlib qt

# file = '/home/zzhang/OneDrive/code/qem/data/STO/adf_average_STO.txt'

# image = np.loadtxt(file)
# file = '/home/zzhang/OneDrive/data/Nico/28.03.2024-STOonSTO21mrad/83-100/aligned_hdf5/1859 HAADF 33.7 nm.emd.hspy'
file = '1859 HAADF 33.7 nm.emd.hspy'
# file ='/home/zzhang/OneDrive/data/Nico/28.03.2024-STOonSTO21mrad/84-100/aligned_hdf5/1932 HAADF 33.7 nm.emd.hspy'

s = hs.load(file)
image = s.data
# image = np.load(file)
# dx= 0.1645429228960236	
dx = s.axes_manager[1].scale *10
# dx = 0.36254824166962796
model=ImageModelFitting(image, dx=dx,units='A')

# get the unique coordinates by row
# input_coordinates = np.unique(input_coordinates, axis=0)

In [ ]:
model.plot()

In [ ]:
model.find_peaks(min_distance=10,threshold_abs=11600,threshold_rel=0.05)

In [ ]:
model.assign_region_label(1)

In [5]:
model.plot_region()

In [ ]:
%matplotlib qt
cif_file = '/home/zzhang/OneDrive/code/qem/data/STO/SrTiO3_mp-5229_conventional_standard.cif'
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'], reciprocal=True)

In [ ]:
model.find_peaks(min_distance=10,threshold_abs=11600,threshold_rel=0.05,region_index=1)
model.map_lattice(cif_file=cif_file,elements=['Sr','Ti'], reciprocal=True,region_index=1)

In [ ]:
from ase.visualize import view
view(model.region_atomic_column[0].lattice)

In [ ]:
model.add_or_remove_peaks(min_distance=10)

In [9]:
model.plot_coordinates(10)

In [ ]:
model.fit_background = True
params = model.init_params()
model.fit_random_batch(params, batch_size=1000, maxiter=50, tol=1e-3, plot=False, step_size=0.01,verbose=False,num_epoch=1)

In [12]:
model.plot_fitting()

In [ ]:
plt.figure()
plt.hist(model.volume[model.atom_types==0], bins=100, alpha=0.5,label='Sr')
plt.hist(model.volume[model.atom_types==1], bins=100, alpha=0.5,label='Ti')
plt.legend()

In [7]:
model.plot_scs(per_element=True)

In [15]:
model.plot_scs_histogram()

In [ ]:
import numpy as np

# Sample array
arr = np.array([2, 3, 2, 1, 3, 1, 4])

# Get unique values, indices of first occurrence, and inverse mapping
unique_values, first_indices, inverse_indices = np.unique(arr, return_index=True, return_inverse=True)

# Create a dictionary to store the indices of all occurrences for each unique value
occurrences = {value: np.where(inverse_indices == i)[0] for i, value in enumerate(unique_values)}

print("Unique values:", unique_values)
print("Indices of first occurrence:", first_indices)
print("Indices of all occurrences for each unique value:", occurrences)


In [ ]:
import numpy as np
from scipy.cluster.hierarchy import fcluster, linkage

# Sample array with values that need tolerance larger than 1
arr = np.array([1, 2, 2.5, 3.2, 10, 10.5, 15])

# Set the desired tolerance (e.g., values within 3 units are considered the same)
tolerance = 3

# Perform hierarchical clustering
Z = linkage(arr[:, None], method='single', metric='euclidean')

# Create clusters based on the tolerance
clusters = fcluster(Z, t=tolerance, criterion='distance')

# Find unique cluster labels and indices of first occurrence
_, unique_indices = np.unique(clusters, return_index=True)

# Get unique values based on clusters
unique_values = arr[unique_indices]

print("Unique values with tolerance:", unique_values)
print("Indices of first occurrence:", unique_indices)
